### Import Statements

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd
import math as mt

### Loading and preparing data

In [2]:
iris = datasets.load_iris()

In [3]:
df = pd.DataFrame(iris.data)
df.columns = ["sl", "sw", 'pl', 'pw']
y = pd.DataFrame(iris.target)

In [4]:
#Function to find label for a value
#if MIN_Value <=val < (m + Mean_Value) / 2 then it is assigned label a
#if (m + Mean_Value) <=val < Mean_Value then it is assigned label b
#if (Mean_Value) <=val < (Mean_Value + MAX_Value)/2 then it is assigned label c
#if (Mean_Value + MAX_Value)/2 <=val <= MAX_Value  then it is assigned label d

def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

#Function to convert a continuous data into labelled data
#There are 4 lables  - a, b, c, d
def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df[old_feature_name].max()
    third = (maximum + second)/2
    return df[old_feature_name].apply(label, args= (first, second, third))

In [5]:
#Convert all columns to labelled data
df['sl_labeled'] = toLabel(df, 'sl')
df['sw_labeled'] = toLabel(df, 'sw')
df['pl_labeled'] = toLabel(df, 'pl')
df['pw_labeled'] = toLabel(df, 'pw')
df

,sl,sw,pl,pw,sl_labeled,sw_labeled,pl_labeled,pw_labeled
0,5.1,3.5,1.4,0.2,b,c,a,a
1,4.9,3.0,1.4,0.2,a,b,a,a
2,4.7,3.2,1.3,0.2,a,c,a,a
3,4.6,3.1,1.5,0.2,a,c,a,a
4,5.0,3.6,1.4,0.2,a,c,a,a
...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,c,b,c,d
146,6.3,2.5,5.0,1.9,c,a,c,d
147,6.5,3.0,5.2,2.0,c,b,c,d
148,6.2,3.4,5.4,2.3,c,c,d,d


In [6]:
df.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)

In [7]:
set(df['sl_labeled'])

{'a', 'b', 'c', 'd'}

### Helper functions

In [8]:
# function to check if list y contains only one class
#return true if no. of elements(unique) in set is 1
# y is pd dataframe so first convert it to numpy array
def ispure(y):
    return len(set(y.values.reshape(-1))) == 1

In [9]:
#Returns a set of counts of various classes present in y
def class_counts(y):
    counts = {}  #initialize dict
    for c in y.values.reshape(-1):
#     for c in y:
        if c not in counts:   #if class 'c' label not already in dict
            counts[c] = 0 
        counts[c] += 1
    return counts

In [10]:
#calculates entropy/Info req
# def entropy(y):
#     k = len(y)  #total no. of classes
#     entropy = float(0)
#     counts = class_counts(y) #dict
#     for cls in counts:  #for each unique class add -pi * log(pi) where pi = probab of ith class
#         entropy -= (counts[cls]/k)*(mt.log(counts[cls]/k))
#     return entropy

In [11]:
class_counts(df.pl_labeled)
t = df['pl_labeled'] == 'a'
# y[t]

In [12]:
# # split df(original dataframe of training set) 
# # and y on basis of wether value of feature f in df
# # belongs to split_val class or not
# def split_data(df, y, f, split_val):
#     new_df = df.copy()  
#     new_y = y.copy()
#     X_f = (df[f] == split_val) # X_f will have True where featuer f == split_val class 
#     l1 = []
#     l2 = []
#     # l1 and l2 are lists which will store indices
#     # to be dropped from new_y and new_df respectively
    
#     # Drop all rows which don't have f == k
#     for i in range(len(X_f)):
#         if X_f[X_f.index[i]] == False:
#             l1.append(new_y.index[i])
#             l2.append(new_df.index[i])
#     new_y.drop(l1, inplace=True)
#     new_df.drop(l2, inplace=True)
#     return new_df,new_y

In [13]:
# alternate approach for split_data
def split_data(df, y, f, split_val):
    X_f = df[f] == split_val
    return df[X_f], y[X_f]

In [14]:
# demo of split_data()
xx, yy = split_data(df, y, 'sl_labeled', 'a')
print(type(xx), type(yy))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


### Object Metrices (Accuracy and Info gain)

In [15]:
# alternate approach for entropy w/o using class_counts()
def entropy(y):
    _, counts = np.unique(y, return_counts=True)
    probab = counts / sum(counts)
    return sum(probab * -np.log(probab))

In [16]:
#demo of entropy for dataFrame "y"
entropy(y)

1.0986122886681096

In [17]:
# Calc Info gain = Info_D - sum(Info_Di)
# curr_f --> currently selected feature
def gain(df, y, curr_f):
    entropy_f = float(0)
    for k in set(df[curr_f]):   #for all unique classes have a split
        _, new_y = split_data(df, y, curr_f, k)
        entropy_f += (len(new_y)/len(y)) * entropy(new_y)
        
    return entropy(y) - entropy_f

In [18]:
# demo of gain()
print(gain(df, y, 'pw_labeled'))
gain(df, y, 'sw_labeled')

0.8752583089296135


0.2066331167565849

In [19]:
# if we predict the most common y as the output
# find sum of all these mistakes = total - correctly predicted
def calcMistakes(y):
#     t = (class_counts(y).values())
    _, t = np.unique(y, return_counts=True)
    return sum(t) - max(t)

In [20]:
calcMistakes(y)

100

In [21]:
# loop over possible values : val
# find subset of df & y with f == val
# find number of mistakes in this subset 
# if we predict the most common y as the output
# find sum of all these mistakes thru "calcMistakes()"
def accuracy(df, y, curr_f):
#     X = df[curr_f]    # X will store curr feature's values from our training dataset
    tot_mistakes = 0
    for k in set(df[curr_f]):  #set results in all possible_values of elements in df for curr feature
        _, new_y = split_data(df, y, curr_f, k)
        tot_mistakes += calcMistakes(new_y)
    return 1 - tot_mistakes/len(y)

In [22]:
# demo of accuracy function
accuracy(df, y, "sl_labeled")

0.6266666666666667

### More helper functions

In [23]:
# This function finds best feature to split on
#Can use accuracy or Info Gain
def find_best_split(df, y, unused_features):
    best_obj_metric = 0
    for f in unused_features:
        # Calc accuracy if we split on feature f
        # Uncomment want accuracy as deciding factor
#         curr_obj_metric  = accuracy(df, y, f) 
#         print("Accuracy ", f, curr_obj_metric)

        # Calc Info Gain for feature f
        curr_obj_metric = gain(df, y, f)
#         print("Gain ", f, curr_obj_metric)
        
        # update best feature so that that particular feature
        # makes least number of mistakes
        if curr_obj_metric >= best_obj_metric:
            best_feature = f
            best_obj_metric = curr_obj_metric
    return best_feature

In [24]:
# This function classifies data as maximum occuring element
def classify_data(y):
    counts = class_counts(y)
    return max(counts, key=counts.get)    

In [25]:
# ignore
tt = {0: 23, 1: 52, 2:12}
max(tt, key=tt.get)

1

### Print Steps

In [26]:
# function to print steps
def print_steps(df, y, unused_features, level):
    print("Level ", level)
    print("unused_features: ", unused_features)
    #base case
    # 1. unused is empty        
    # 2. y contains only one distinct value
#     print("------------------------------------------------------->", level, len(y),)
    if ispure(y) or not len(unused_features):
        print("Total: ", len(y))
        counts = class_counts(y)
        for k in counts:
            print("Count of", k, "= ", counts[k])
        print("Current Entropy is ", entropy(y))
        print("Data Classified as: ", classify_data(y))
        print("|| Reached Leaf Node ||")
        print()
        return
    
    best_feature = find_best_split(df, y, unused_features)
    print("Gain: ", gain(df, y, best_feature))
        
    # here you should know the best feature
    # print it out
    print("Best Feature ", best_feature)
    print()
    
    # remove best feature from unused features
    unused_features.remove(best_feature)
    # loop over possible values of best feature
    # call build tree recursively
    possible_vals = set(df[best_feature])
    for split_val in possible_vals:
        new_df, new_y = split_data(df, y, best_feature, split_val)
#         print(type(new_df), type(new_y))
        print("Split for ", best_feature, "== ", split_val)
        print("----------------------------------")
        print_steps(new_df, new_y, unused_features, level + 1)

In [27]:
# y = pd.DataFrame(iris.target)
unused_features = set(df.columns)
print_steps(df, y, unused_features, 0)

Level  0
unused_features:  {'sl_labeled', 'sw_labeled', 'pl_labeled', 'pw_labeled'}
Gain:  0.8752583089296135
Best Feature  pw_labeled

Split for  pw_labeled ==  a
----------------------------------
Level  1
unused_features:  {'sl_labeled', 'sw_labeled', 'pl_labeled'}
Total:  50
Count of 0 =  50
Current Entropy is  0.0
Data Classified as:  0
|| Reached Leaf Node ||

Split for  pw_labeled ==  c
----------------------------------
Level  1
unused_features:  {'sl_labeled', 'sw_labeled', 'pl_labeled'}
Gain:  0.21536778951600982
Best Feature  pl_labeled

Split for  pl_labeled ==  c
----------------------------------
Level  2
unused_features:  {'sl_labeled', 'sw_labeled'}
Gain:  0.10945355973980492
Best Feature  sl_labeled

Split for  sl_labeled ==  a
----------------------------------
Level  3
unused_features:  {'sw_labeled'}
Total:  1
Count of 2 =  1
Current Entropy is  0.0
Data Classified as:  2
|| Reached Leaf Node ||

Split for  sl_labeled ==  c
----------------------------------
Level  

### Creating nodes for trees

In [28]:
# leaf node of tree that classifies data
# and has its dataset
class LeafNode:
    def __init__(self, df, y):
        self.x = df
        self.y = y
        self.prediction = classify_data(y)  # assigning max occuring value

In [29]:
# DecisionNode asks question according to which previous node
# is to be splitted
# branches will store all possible paths due to splitting as child nodes
class DecisionNode:
    def __init__(self,df, y, feature, branches):
        self.x = df
        self.y = y
        self.question = feature
        self.branches = branches

### Building and printing tree

In [30]:
def build_tree(df, y, unused_features):
    # base case
    # classify data with leaf node
    if ispure(y) or not len(unused_features):
        return LeafNode(df, y)
    
    # find best feature to split
    best_feature = find_best_split(df, y, unused_features)
    unused_features.remove(best_feature)
    branches = []
    
    # for all possible vals of best_feature 
    # 1.split data
    # 2.build tree of splitted data
    # 3.append built tree to branches list
    possible_vals = set(df[best_feature])
    for split_val in possible_vals:
        new_df, new_y = split_data(df, y, best_feature, split_val)
        branches.append(build_tree(new_df, new_y, unused_features))
    
    # return decision node with best_f as question and
    # built trees as branches
    return DecisionNode(df, y, best_feature, branches)    

In [31]:
unused_features = set(df.columns)
tree = build_tree(df, y, unused_features)

In [32]:
# demo of built tree
print(tree.branches)
# tree.branches[].branches[0].prediction

[<__main__.LeafNode object at 0x7fd18e672a90>, <__main__.DecisionNode object at 0x7fd18e606a10>, <__main__.LeafNode object at 0x7fd18e606510>, <__main__.LeafNode object at 0x7fd18e606550>]


In [33]:
def print_tree(root, spacing=""):
    # base case : reached leaf node
    if isinstance(root, LeafNode):
        print(spacing + "Reached Leaf Node")
        print(spacing, class_counts(root.y)) #printing counts of all classes present
        print(spacing + "Prediction: ", root.prediction)
        print()
        return
    
    print(spacing + "Question/Decision_Parameter: ", root.question+ "??")
    split_col = root.question
    print(spacing + "Gain: ", gain(root.x, root.y, split_col))
    spacing += "      |"
    # call recursively for printing tree on child_nodes in branches
    for child_node in root.branches:
        print(spacing + " == ", child_node.x[split_col].iloc[0]) #print branch of decision param.
        print_tree(child_node, spacing) 
        print()

In [34]:
print_tree(tree)

Question/Decision_Parameter:  pw_labeled??
Gain:  0.8752583089296135
      | ==  a
      |Reached Leaf Node
      | {0: 50}
      |Prediction:  0


      | ==  c
      |Question/Decision_Parameter:  pl_labeled??
      |Gain:  0.21536778951600982
      |      | ==  c
      |      |Question/Decision_Parameter:  sl_labeled??
      |      |Gain:  0.10945355973980492
      |      |      | ==  a
      |      |      |Reached Leaf Node
      |      |      | {2: 1}
      |      |      |Prediction:  2


      |      |      | ==  c
      |      |      |Question/Decision_Parameter:  sw_labeled??
      |      |      |Gain:  0.061051894017797426
      |      |      |      | ==  a
      |      |      |      |Reached Leaf Node
      |      |      |      | {1: 3, 2: 1}
      |      |      |      |Prediction:  1


      |      |      |      | ==  c
      |      |      |      |Reached Leaf Node
      |      |      |      | {1: 6}
      |      |      |      |Prediction:  1


      |      |      |      | =